In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

def prepare_data_iris():
    data = load_iris()
    X = data.data
    y = LabelEncoder().fit_transform(data.target)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
    # scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def prepare_data_heart():
    df = pd.read_csv("heart.csv", na_values='?')
    df.dropna(inplace=True)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    y = (y > 0).astype(int)  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
    # scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def prepare_data_mnist():
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    X_train = X_train.reshape(-1, 28*28) / 255.0
    X_test = X_test.reshape(-1, 28*28) / 255.0
    return X_train, X_test, y_train, y_test

In [6]:
class FFNN(nn.Module):
        def __init__(self):
            super(FFNN, self).__init__()
            self.fc1 = nn.Linear(input_size, 64)
            self.fc2 = nn.Linear(64, 32)
            self.fc3 = nn.Linear(32, num_classes)
            self.tanh = nn.Tanh()
            self.softmax = nn.Softmax(dim=1)
        
        def forward(self, x):
            x = self.tanh(self.fc1(x))
            x = self.tanh(self.fc2(x))
            x = self.fc3(x)
            return self.softmax(x)
def ffnn_pytorch(X_train, X_test, y_train, y_test, input_size, num_classes):    
    model = FFNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    for epoch in range(100):
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = (predicted == y_test_tensor).float().mean()
        print(f'PyTorch Model Accuracy: {accuracy.item() * 100:.2f}%')

In [7]:
for dataset, prepare_function, num_classes, input_size in zip(
    ['Iris', 'Heart Disease', 'MNIST'],
    [prepare_data_iris, prepare_data_heart, prepare_data_mnist],
    [3, 2, 10],
    [4, 13, 28*28]
):
    print(f"\nRunning FFNN on {dataset} dataset:")
    X_train, X_test, y_train, y_test = prepare_function()
    print("Using PyTorch:")
    ffnn_pytorch(X_train, X_test, y_train, y_test, input_size, num_classes)


Running FFNN on Iris dataset:
Using PyTorch:
PyTorch Model Accuracy: 96.67%

Running FFNN on Heart Disease dataset:
Using PyTorch:
PyTorch Model Accuracy: 74.07%

Running FFNN on MNIST dataset:
Using PyTorch:
PyTorch Model Accuracy: 95.59%
